In [49]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [50]:
behaviours_data = pd.read_pickle('behaviors_encoded_data.df')#pd.read_csv('behaviours_encoded_data.csv')

news_data = pd.read_csv('news_with_timestamp.csv')

behaviours_data_train, behaviours_data_test = train_test_split(behaviours_data, test_size=0.01, random_state=42)


In [11]:
articles = news_data['Abstract']
user_history = test_set['History']

print(user_history)

155965    [20695, 28533, 15555, 3502, 30819, 21466, 1000...
155966    [24706, 23640, 4032, 15396, 18632, 13556, 2860...
155967    [1402, 7385, 25550, 3289, 6677, 28808, 18296, ...
155968    [7191, 249, 28366, 16615, 15914, 15267, 21495,...
155969                         [14081, 13826, 31684, 23682]
                                ...                        
156960    [21177, 19890, 2655, 14201, 16229, 21716, 1626...
156961    [27147, 6337, 19238, 6950, 7630, 20287, 8013, ...
156962    [19160, 26885, 30516, 27147, 22578, 9314, 1336...
156963    [11410, 15584, 2260, 3681, 31665, 1131, 16301,...
156964                                       [18448, 14580]
Name: History, Length: 1000, dtype: object


In [4]:
# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the articles
tfidf_matrix = tfidf_vectorizer.fit_transform(articles.values.astype('U'))

In [5]:
# Calculate cosine similarity between articles
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [12]:
def recommend_articles(user_article_indices, cosine_sim_matrix, num_recommendations=5):
    # Collecting similarity scores for articles in user_article_indices
    sim_scores = {}
    for idx in user_article_indices:
        for i, score in enumerate(cosine_sim_matrix[idx]):
            if i not in user_article_indices:  # Skip already interacted articles
                if i in sim_scores:
                    sim_scores[i] += score
                else:
                    sim_scores[i] = score

    # Sorting articles based on similarity scores
    recommended_article_indices = sorted(sim_scores, key=sim_scores.get, reverse=True)[:num_recommendations]

    return recommended_article_indices

In [53]:
user_article_indices =[23642, 4852, 27685, 31188]
recommended_article_indices = recommend_articles(user_article_indices, cosine_sim)
print(recommended_article_indices)

[19886, 3368, 40550, 5313, 39351]


In [42]:
# Get recommended_articles: List of lists. Each inner list contains integers of recommended articles for the corresponding user.
recommended_articles_users = []
last_clicks = test_set.drop_duplicates(subset='User ID', keep='last')
data = last_clicks.dropna(subset=['History'], inplace=True)
print(last_clicks.shape)

for userID in last_clicks['User ID']:
    articles = last_clicks[last_clicks["User ID"]==userID]["History"].values
    idxs = recommend_articles(articles[0], cosine_sim)
    recommended_articles_users.append(idxs)

recommended_articles_users



C:\Users\lizzi\AppData\Local\Temp\ipykernel_10308\968042222.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data = last_clicks.dropna(subset=['History'], inplace=True)


(959, 7)
[20695, 28533, 15555, 3502, 30819, 21466, 1000, 18691, 27176, 25326, 25326, 3761, 23913, 2784, 9740, 14216]
[24706, 23640, 4032, 15396, 18632, 13556, 2860, 21479, 31017, 31770, 16666, 7854]
[1402, 7385, 25550, 3289, 6677, 28808, 18296, 6677, 28808, 5857, 8118, 7233, 23662, 8118, 18023, 2675, 17411, 28808, 28707, 29049, 1402, 31856, 2675, 20130, 27927, 17411, 28808, 31698, 29875, 28594, 31684, 21916, 11299, 13995, 22408, 1559, 24237, 20403]
[7191, 249, 28366, 16615, 15914, 15267, 21495, 1003, 45710]
[14081, 13826, 31684, 23682]
[1152, 2390, 919, 31589, 30713, 22198, 22909, 11785, 14216, 7177, 13458, 21963, 23894]
[5641, 21777, 16261, 13937, 14248, 17437, 18351, 29944, 17088, 21141, 25858, 7372, 3242, 29461, 31482, 22090, 8920, 1366, 14549, 25965, 13729, 5994, 1559, 6522, 28174, 25180, 24135, 28447, 14580, 17662, 19211, 30688, 16033, 2504, 9284, 17576, 20447, 17118, 26272, 22367, 37949, 46852]
[26964, 11870, 1060, 742, 4362, 6312, 7325, 15037, 14081, 6892, 12060, 28505, 25795, 2

[31858, 18267, 1649, 29855, 18988, 9646, 31568, 21515, 2439, 19097, 21242, 23684, 30470, 20252, 2827, 31183, 14425, 9869, 28841, 27427, 26883, 21373, 12283, 29049, 9473, 17714, 2357, 25342, 13167, 31471, 19633, 5041, 7390, 31638, 14023, 19768, 8005, 11719, 24357, 21483, 10569, 16213, 17469, 31719, 31777, 27262, 17071, 31309, 21341, 3732, 30547, 26686, 20175, 26686]
[22886, 873, 9218, 6932, 27470, 3521, 4291, 3153, 4291, 4064, 15207, 8155, 18720, 22261, 22245, 24517, 29198, 31824, 9841, 11269, 20591, 19818, 9999, 15190, 10313, 7956, 16798, 3613, 22646, 1575, 24488, 29157, 6454, 8318, 20474, 18447, 15381, 26637, 945, 4984, 13918, 30370, 18908, 30298, 12096, 1498, 21984, 21399, 21518, 11465, 3761, 24597, 31130, 11302, 386, 13458, 17373, 19940, 6232, 23130, 16405, 21080, 20301, 4797, 17796, 7936, 18192, 16265, 35582, 33677, 36100, 47380, 39981, 47399, 50720, 41712, 36541]
[18558, 23640, 4852, 29573, 31017, 12283, 10024, 2012, 9291, 11948, 49499]
[17205, 18151, 16666]
[24357, 19718, 13849, 

[22288, 20695, 301, 16639, 2393, 2706, 8657, 16000, 9049, 13910, 20778, 16356, 28044, 12161, 18691, 15546, 28751, 16859, 31413, 28532, 3276, 9690, 2190]
[13432, 4079, 7404, 15002, 4129, 31824, 20841, 31482, 22416, 21466, 7015, 31381, 26951, 14216, 19140, 10314, 46316]
[18173, 3559, 13153, 27385, 288, 11465, 6694, 29283, 22640, 9942, 22555]
[6892, 10545, 28879, 24677, 16247, 31552, 9930, 22244, 14205, 3873, 4822, 16666, 19764, 21667, 27048, 16248, 26791, 28360, 14216, 23850, 16977, 11159, 818, 29686, 24953, 31170, 17396, 21073, 5965, 31347, 21341, 26272, 18520, 23123, 25404]
[28062, 25465, 20738, 30744, 11933, 2393, 20003, 12991, 14097, 17699, 4852, 27385, 23902, 31577, 31620, 14166, 7614, 3459, 5344]
[13985, 14309, 15207, 18076, 6067, 16757, 6892, 30744, 19704, 23115, 9692, 13540, 29508, 15659, 30142, 8650, 13953, 22630, 17935, 20547, 2077, 28018, 22597, 8396, 25075, 13428, 11159, 23701, 16173, 20912, 17677, 44834]
[24013, 6344, 14564, 825, 9621, 21974, 10083, 3662, 28905, 25176, 6246,

[6299, 6067, 8232, 22728, 5865, 21620, 1649, 616, 6360, 31692, 21222, 29926, 24104, 5010, 31421, 2566, 13521, 5929, 7176, 31429, 7074, 4241, 8780, 2827, 21466, 15723, 24779, 23600, 5223, 30142, 20080, 22849, 20719, 23420, 9818, 4412, 6766, 3420, 1614, 20547, 21518, 9329, 2077, 1724, 27467, 26513, 14216, 6247, 17430, 16405, 25092, 24311, 44834]
[17092]
[7657, 12850, 3459, 13409, 25924, 17160]
[3242]
[26726, 18985, 11250, 1814, 19619, 19548, 16025, 14699, 5424, 3079, 31824, 3046, 29198, 23477, 7422, 18988, 4852, 22646, 26064, 12671, 7796, 21141, 28620, 9769, 9833, 18275, 12844, 19695, 19004, 6874, 6874, 19863, 20287, 21984, 1536, 16510, 4076, 20357, 19143, 20895, 17662, 9278, 36404, 34210, 49195]
[86, 9218, 10611, 11404, 9769, 18803, 27927, 14653, 31283, 4044, 15254, 27736, 21554, 2043, 21752, 37296, 33951, 34599]
[5603, 25365, 12991, 27101]
[5078, 15276, 28919, 26544, 6360, 31750, 7434, 12155, 5182, 5867, 10684, 3549, 9469, 16247, 31668, 26787, 16335, 22765, 22127, 21466, 24371, 21018, 

...

[11870, 20840, 12213, 9701, 22416, 9227, 31780, 25836, 23684, 15322, 17437, 11081, 27867, 28805, 22255, 2077, 8682, 19863, 20740, 21114, 20309, 10756, 16050, 12850, 9942, 11211, 3732, 31347, 31309, 44834, 32953, 15303]
[20896, 3848, 9016, 12898, 19084, 13635, 30642, 5309, 2603, 6409, 20265, 17537, 800, 3806, 2087, 30470, 9922, 23377, 811, 20841, 22024, 7023, 27147, 27867, 800, 22892, 16355, 800, 22805, 4043, 11836, 4076, 1358, 9309, 22909, 11395, 29990, 10246, 8509, 13121, 26187, 25873, 22534, 31288, 6014, 31942, 198, 20441, 25186, 31414]
[22886, 4258, 31501, 25142, 29504, 7523, 15555, 1515, 26064, 30757, 384, 9339, 5887, 14824, 7204, 13510, 27417, 28955, 20628, 41489, 48936]
[2469, 2239, 14309, 8155, 15687, 29981, 29926, 2164, 21885, 12300, 10489, 22973, 26885, 9055, 22556, 27867, 22512, 3107, 7279, 6854, 11465, 11464, 14695, 20142, 2058, 31347, 30547, 36920, 32820]
[1165, 20203, 7858, 4185, 22967, 27795, 21772, 29484, 16704, 18967, 42612, 35881, 36876, 40684, 41344, 34329, 33030, 100

[29127, 23822, 26544, 27339, 791, 13910, 24646, 27048, 24501, 14905]
[11836, 23457, 6024, 11465, 27048, 20142, 14216, 31572, 2242, 34839]
[31884, 31942, 16248, 5660, 1452, 15932, 23717, 29974, 18917, 21791, 26298, 27204, 31719, 818, 28360, 15784, 6014, 608, 17796, 36404, 45798, 31790]
[20528, 26054, 17437, 30180, 31416, 3096, 31899, 16308, 13933]
[1776, 5366, 9218, 5117, 24195, 12849, 29185, 14864, 11996, 14309, 12667, 5623, 27904, 2793, 30744, 12060, 2758, 11617, 23613, 23666, 31689, 25550, 18832, 29751, 8368, 8268, 20857, 31651, 31503, 14883, 997, 29926, 24104, 5037, 2898, 26322, 28748, 11709, 24027, 27591, 21018, 22127, 22765, 19702, 22695, 24809, 15659, 30713, 27128, 598, 6078, 26107, 30330, 3107, 26042, 7466, 8560, 8709, 10371, 8560, 14564, 25716, 30371, 4557, 3439, 14313, 27081, 19074, 16810, 30142, 945, 12880, 29642, 358, 28825, 6808, 8107, 26081, 5988, 12880, 9339, 30956, 19700, 27774, 3420, 30103, 9415, 7733, 8000, 22306, 20746, 27676, 8410, 2077, 28018, 28018, 8410, 21518, 27

[27675, 27675, 15687, 13690, 28479, 2711, 19914, 18632, 6649, 7655, 14173, 10327, 13452, 24370, 1396, 22416, 21466, 27128, 23600, 29776, 14070, 15617, 23089, 2077, 4115, 7299, 20707, 11465, 17633, 22909, 19158, 23590, 16666, 11159, 18967, 18448, 24020, 22243, 16835, 30444, 19408, 18976, 25092, 27474, 27719, 23210, 27798, 22485, 27271, 10383, 22472, 11615, 10383, 7340, 13409, 23259, 30541, 28661, 14346, 22247, 16349, 17677, 22642, 22367, 48392, 40564]
[26787, 27147, 26637, 23913, 11308, 49976]
[11870, 17638, 31952, 27760, 28170, 8671, 6892, 22728, 28505, 27666, 7793, 10198, 6782, 17205, 20804, 24513, 3461, 31627, 21777, 22416, 22973, 23179, 23898, 18023, 17605, 24779, 24297, 5341, 13613, 20041, 26637, 7436, 7630, 7630, 6950, 1614, 11653, 2077, 27606, 28257, 25225, 28981, 1536, 24357, 23913, 11983, 13458, 18976, 25894, 4236]
[11870, 2758, 2439, 7054]
[18274, 16468, 19084, 25783, 5426, 14426, 24311, 29787, 30822, 6476, 17891, 22555, 23241, 22280, 21785, 47672]
[6892, 28043, 15614, 4810, 3

[22288, 11025, 6850, 14471, 24264, 1165, 7690, 19084, 9397, 10492, 20804, 86, 4547, 26933, 25872, 1113, 1987, 30642, 9930, 9966, 30789, 29464, 13012, 14173, 6717, 27368, 7951, 31933, 25955, 25363, 14097, 25955, 1252, 1609, 18875, 17437, 27161, 29776, 22967, 24104, 31921, 14352, 14303, 29227, 22967, 16330, 1614, 30545, 22967, 12709, 11653, 8650, 29645, 16620, 4945, 2077, 21518, 24875, 18277, 18313, 18277, 22967, 26518, 30266, 22048, 19090, 22967, 16479, 22909, 9452, 22967, 27048, 6477, 10937, 18185, 14113, 27283, 25651, 13678, 11080, 11080, 8396, 23913, 22967, 21109, 15523, 13413, 14216, 18072, 15587, 25873, 20583, 7867, 11330, 20467, 6014, 3945, 3945, 18074, 2735, 16549, 22967, 20918, 28764, 27553, 22441, 11722, 23871, 25529, 21341, 10569, 7506, 41489, 49499, 40124, 44062, 50232, 22967, 41691]
[11138, 1235, 26787, 23819, 2518, 26885, 25936, 30713, 26513, 27048, 8135]
[6892, 20865, 28505, 5465, 2485, 2087, 4852, 26194, 21466, 24775, 29776, 29461, 25744, 7390, 18691, 11465, 18244, 15501,

[[15935, 30338, 13194, 29934, 21638],
 [34989, 10307, 2221, 45958, 5473],
 [28122, 48698, 24464, 13432, 50071],
 [24740, 18377, 7162, 50072, 9073],
 [31170, 8005, 44900, 257, 21277],
 [38, 41, 133, 137, 175],
 [15228, 50834, 9696, 2920, 123],
 [11897, 9446, 2504, 43096, 2825],
 [24637, 4081, 7161, 13213, 31758],
 [38, 41, 133, 137, 175],
 [18825, 886, 3366, 16922, 46171],
 [18659, 22416, 23564, 13623, 35306],
 [9995, 25885, 19730, 26686, 28348],
 [24327, 27628, 5831, 9995, 26120],
 [86, 22585, 38, 41, 133],
 [6677, 50071, 40347, 38, 41],
 [38, 41, 133, 137, 175],
 [28348, 11875, 50834, 19417, 10148],
 [86, 9552, 6915, 19417, 27914],
 [1439, 17560, 20265, 29271, 397],
 [6654, 40002, 21666, 25765, 15408],
 [27881, 18245, 27093, 7260, 12274],
 [38, 41, 133, 137, 175],
 [4409, 8231, 13663, 30317, 27140],
 [28978, 17893, 48417, 6575, 3938],
 [25344, 28470, 26269, 34232, 36714],
 [15659, 46231, 24185, 13404, 10235],
 [38, 41, 133, 137, 175],
 [123, 1439, 22924, 31743, 31498],
 [30317, 14054,

In [47]:
len(recommended_articles_users)

959

In [43]:
behaviors_data_users = []
for i in range(len(last_clicks['User ID'])):
    behaviors_data_users.append([last_clicks['User ID'].iloc[i]])
    behaviors_data_users[i].append(last_clicks['History'].iloc[i])
    
len(behaviors_data_users)

959

In [44]:

def calculate_mrr_from_recommendations(behaviors_data, recommended_articles):
    """
    Calculate Mean Reciprocal Rank (MRR) for a set of user interactions and recommendations.
 
    Parameters:
    - behaviors_data: List of tuples/lists. Each tuple/list should contain:
                      [integer user ID, list of article interactions as integers]
    - recommended_articles: List of lists. Each inner list contains integers of recommended articles for the corresponding user.
 
    Returns:
    - MRR (float): The mean reciprocal rank of the first relevant recommendation.
    """
    reciprocal_ranks = []
 
    for (user, history), recommendations in zip(behaviors_data, recommended_articles):
        # History is already a set of integers
        history_set = set(history)
 
        # Check for the first occurrence of any article from the user's history in the recommended list
        rank = next((1 + idx for idx, article in enumerate(recommendations) if article in history_set), None)
        # If there's at least one history article in the recommendations, calculate its reciprocal rank
        if rank is not None:
            reciprocal_ranks.append(1 / rank)
    # Calculate the mean of the reciprocal ranks
    if reciprocal_ranks:
        mrr = sum(reciprocal_ranks) / len(reciprocal_ranks)
    else:
        mrr = 0
 
    return mrr




In [46]:
calculate_mrr_from_recommendations(behaviors_data_users, recommended_articles_users)

0